# Recuperación de información mediante Self Query Retriever

In [20]:
!pip install langchain-community
!pip install chromadb

  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.4 which is incompatible.


In [33]:
!pip show langchain
!pip show langchain-community
!pip install sentence-transformers

Name: langchain
Version: 0.2.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: c:\users\emolt\anaconda3\lib\site-packages
Requires: PyYAML, requests, aiohttp, tenacity, pydantic, langchain-text-splitters, async-timeout, numpy, langsmith, SQLAlchemy, langchain-core
Required-by: langchain-community
Name: langchain-community
Version: 0.2.19
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: c:\users\emolt\anaconda3\lib\site-packages
Requires: requests, numpy, PyYAML, dataclasses-json, tenacity, langchain-core, langchain, aiohttp, langsmith, SQLAlchemy
Required-by: 
  Using cached numpy-1.22.4-cp38-cp38-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninsta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires numpy>=1.22.5, but you have numpy 1.22.4 which is incompatible.


In [1]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [1]:
local_llm = "llama3.2:latest"
#url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION
url_llm = "http://localhost:11434"
from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 


C:\Users\emolt\AppData\Local\Temp\ipykernel_13292\670553702.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=local_llm, base_url=url_llm, temperature=0)


In [2]:
import csv
from typing import Dict, List, Optional
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document

class CSVLoader(BaseLoader):
    def __init__(
        self,
        file_path: str,
        source_column: Optional[str] = None,
        metadata_columns: Optional[List[str]] = None,
        csv_args: Optional[Dict] = None,
        encoding: Optional[str] = None,
    ):
        self.file_path = file_path
        self.source_column = source_column
        self.encoding = encoding
        self.csv_args = csv_args or {}
        self.metadata_columns = metadata_columns or []

    def load(self) -> List[Document]:
        docs = []
        with open(self.file_path, newline="", encoding=self.encoding) as csvfile:
            csv_reader = csv.DictReader(csvfile, **self.csv_args)
            for i, row in enumerate(csv_reader):
                metadata = {"row": i}
                for col in self.metadata_columns:
                    if col in row:
                        metadata[col] = row[col].strip()
                content = []
                for k, v in row.items():
                    if k != self.source_column and k not in self.metadata_columns:
                        content.append(f"{k.strip()}: {v.strip()}")
                doc_content = "\n".join(content)
                doc = Document(page_content=doc_content, metadata=metadata)
                docs.append(doc)

        return docs

In [3]:
metadata_columns = ["row", "grant agreement", "project acronym", "organisation PIC", "organisation name", "Small or Medium-sized Enterprise",
                   "organisation activity type", "organisation country", "organisation role", "funding for the organisation", 
                    "organisation total participation cost", "project title", "project total cost", "funding for the project",
                   "master call", "subcall", "type of proposal", "project legal basis", "project topic"]

# Instancia el CSVLoader con el archivo CSV y las columnas de metadatos
loader = CSVLoader(
    file_path = r"C:\Users\emolt\OneDrive - UMH\MASTER\TFM\BASE\cordis_data_processed.csv",
    source_column= None,  # Opcional: columna para establecer como origen
    metadata_columns=metadata_columns,
    encoding="latin1"
)

# Carga los documentos del CSV
raw_documents = loader.load()

In [4]:
raw_documents[0]

Document(metadata={'row': '0', 'grant agreement': '964203', 'project acronym': 'LINkS', 'organisation PIC': '891131823', 'organisation name': 'MICRONIT BV', 'Small or Medium-sized Enterprise': 'True', 'organisation activity type': 'PRC', 'organisation country': 'NL', 'organisation role': 'participant', 'funding for the organisation': '451125.0', 'organisation total participation cost': '451125.0', 'project title': 'Long-range electrodynamic INteractions between proteinS', 'project total cost': '3090575.0', 'funding for the project': '3090575.0', 'master call': 'H2020-FETOPEN-2018-2020', 'subcall': 'H2020-FETOPEN-2018-2019-2020-01', 'type of proposal': 'RIA', 'project legal basis': 'H2020-EU.1.2. EXCELLENT SCIENCE - Future and Emerging Technologies (FET)', 'project topic': 'FETOPEN-01-2018-2019-2020 FET-Open Challenging Current Thinking'}, page_content='project objective: LINkS will change the paradigm of the self-organization of the intracellular living matter by demonstrating the exis

In [5]:
len(raw_documents)

177640

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=400)
documents = text_splitter.split_documents(raw_documents) ### RECORTE DE LA BASE DE DATOS 

In [7]:
# SE USA UNA DE LAS GPU
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cpu"} # Al no tener NVIDIA es necesario cambiarlo model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

C:\Users\emolt\AppData\Local\Temp\ipykernel_13292\3381727767.py:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
C:\Users\emolt\anaconda3\envs\py310real\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="row",
        description="The row in which the data is located.",
        type="int",
    ),
    AttributeInfo(
        name="grant agreement",
        description="The grant agreement number of the project, which is a identificator of the project.",
        type="int",
    ),
    AttributeInfo(
        name="project acronym",
        description="The acronym of the proyect, which is its shorted name.",
        type="string",
    ),
    AttributeInfo(
        name="organisation PIC",
        description="The Participant Identification Code, which is a identificator of the organisation.",
        type="int",
    ),
    AttributeInfo(
        name="organisation name", 
        description="The name of the orgnanisation.", 
        type="string"
    ),
    AttributeInfo(
        name="Small or Medium-sized Enterprise", 
        description="Characteristic of the organisation related to the number of employees it has. It can be: True, False or Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation activity type", 
        description="The kind of activity that the organisation carries out. It can be: PRC, REC, HES, OTH, PUB or Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation country", 
        description="The country in which the organisation is established. It is codified in ISO 3166-1. It can also take the value Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation role", 
        description="The role that the organisation has played in the project. It can be: participan', thirdParty, coordinator, partner or internationalPartner",
        type="string"
    ),
    AttributeInfo(
        name="funding for the organisation", 
        description="The amount of money that the organisation has received from the European Union in order to conduct the project.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation total participation cost", 
        description="The amount of money that the organisation has invested in carring out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project title", 
        description="The name of the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project total cost", 
        description="The total amount of money that all the participants in the project have spent to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="funding for the project", 
        description="The total amount of money that the European Union has provided to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="funding for the project", 
        description="The total amount of money that the European Union has provided to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="master call", 
        description="The master call within which the project has been framed.", 
        type="string"
    ),
    AttributeInfo(
        name="subcall", 
        description="The subcall within which the project has been framed.", 
        type="string"
    ),
    AttributeInfo(
        name="type of proposal", 
        description="The kind of proposal that constitutes the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project legal basis", 
        description="The kind of proposal that constitutes the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project topic", 
        description="The topic within which the project has been framed.", 
        type="string"
    )
]
document_content_description = "Brief summary of aspects corresponding to the participation of an organisation in a determined research project funded by the European Union within the Horison 2020 programme."

In [10]:
!pip show chromadb

Name: chromadb
Version: 0.5.23
Summary: Chroma.
Home-page: None
Author: None
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: None
Location: c:\users\emolt\anaconda3\lib\site-packages
Requires: chroma-hnswlib, rich, importlib-resources, tenacity, typer, tqdm, bcrypt, typing-extensions, tokenizers, overrides, opentelemetry-api, orjson, pypika, pydantic, mmh3, onnxruntime, kubernetes, numpy, opentelemetry-sdk, uvicorn, build, PyYAML, opentelemetry-exporter-otlp-proto-grpc, posthog, fastapi, opentelemetry-instrumentation-fastapi, grpcio, graphlib-backport, httpx
Required-by: langchain-chroma


In [11]:
import sys
print(sys.version)

3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]


In [10]:
from langchain.vectorstores import Chroma
import shutil
vectorstore_load = Chroma(persist_directory="./chroma_db_5", embedding_function= emb)

C:\Users\emolt\AppData\Local\Temp\ipykernel_13292\85159252.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_load = Chroma(persist_directory="./chroma_db_5", embedding_function= emb)


In [11]:
import os
os.environ["ANONYMIZED_TELEMETRY"] = "False"  # Evita errores con posthog

In [12]:
!pip uninstall lark -y  # Elimina si existe una versión corrupta
!pip install lark --upgrade

ERROR: Invalid requirement: '#'


^C


In [ ]:
!pip show lark

In [13]:
import sys
print(sys.executable)  # Verifica la ruta de Python que se está usando

C:\Users\emolt\anaconda3\envs\py310real\python.exe


In [14]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore_load,
    document_content_description,
    metadata_field_info,
    verbose = True
)

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks and an expert in research projects funded by the European Union under the Horizon 2020 programme.
Use the following context from Horizon 2020 projects to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

In [18]:
import time

inicio = time.time()

#retrieved_docs_1 = retriever.get_relevant_documents(query_1)
answer_1 = rag_chain.invoke(query_1)

fin = time.time()  # Marca de tiempo al final

tiempo_ejecucion = fin - inicio  # Calcular el tiempo de ejecución

print(f"El tiempo de ejecución fue: {tiempo_ejecucion} segundos")

El tiempo de ejecución fue: 114.69579911231995 segundos


In [19]:
answer_1

"I don't have information on a specific Horizon 2020 project with grant agreement 740934. I can try to search for more context or details about the project, but without further information, I couldn't find any relevant data. If you provide more context or details about the project, I'll do my best to help."

In [20]:
import langchain

langchain.debug = False

retriever.invoke(query_1)

[]

In [21]:
retriever.invoke(query_2)

[]

In [31]:
retriever.invoke(query_3)

[]

In [22]:
retriever.invoke(query_4)

OutputParserException: Parsing text
```json
{
    "query": "coordinator",
    "filter": "eq(\"organisation role\", \"coordinator\") and eq(\"grant agreement\", \"777998\")"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 40.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [25]:
retriever.invoke("¿Qué es inteligencia artificial?")

[]

In [33]:
retriever.invoke(query_5)

KeyboardInterrupt: 

In [23]:
retriever.invoke(query_6)

[]

In [24]:
retriever.invoke(query_7)

OutputParserException: Parsing text
```json
{
    "query": "proposal",
    "filter": "eq(\"project legal basis\", \"grant\") and eq(\"grant agreement\", \"814416\")"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 36.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [26]:
retriever.invoke(query_8)

OutputParserException: Parsing text
### Structured Request Schema
```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

### Example 1 Response
```json
{
    "query": "teenager love",
    "filter": "and(or(eq(\"artist\", \"Taylor Swift\"), eq(\"artist\", \"Katy Perry\")), lt(\"length\", 180), eq(\"genre\", \"pop\"))"
}
```

### Example 2 Response
```json
{
    "query": "",
    "filter": "NO_FILTER"
}
```

### Example 3 Response
```json
{
    "query": "",
    "filter": "eq(\"project_acronym\", \"G9NIGHT\") and eq(\"project_legal_basis\", \"Horizon 2020\")"
}
```
 raised following error:
Got invalid JSON object. Error: Expecting value: line 2 column 14 (char 15)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [27]:
retriever.invoke(query_9)

[]

In [ ]:
retriever.invoke(query_10)

In [ ]:
retriever.invoke(query_11)

In [ ]:
retriever.invoke(query_12)

In [ ]:
retriever.invoke(query_13)

In [ ]:
retriever.invoke(query_14)

In [25]:
import langchain

langchain.debug = True

retriever.invoke(query_20)

[chain/start] [retriever:Retriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
}
[chain/start] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
}
[chain/end] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:query_constructor > llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object format

[]